In [1]:
################## 全局变量
# 数据路径
dir_path = "D:\\hadoop_data\\event_data\\2"
# 区分事件数据的index
event_indexs = [0, 3, 4,6,7, 8] # 事件类型 鼠标点击类型 鼠标点击按键 键盘点击类型 键盘输入 组件名
# prefix db 最大长度
max_seq_length = 100 
# 过长序列长度
same_seq_length = 5

In [ ]:
from prefixspan import PrefixSpan
import os
import pandas as pd
import json

# 读取行为原始数据 [][][]
def load_data():
    paths = os.listdir(dir_path)
    #print(paths)
    data = []
    for idx in range(len(paths)):
        path = paths[idx]
        strs = path.split("_")
        if len(strs) != 2:
            return
        # print("path=", path)
        # try:
        df = pd.read_csv(dir_path + "/" + path)
        df = df.fillna("")
        data.append(df.values)
        # except:
        #     print("error path=", path)
        #     os.remove(dir_path + "/" + path)
    return data
event_data = load_data()
print(event_data)

In [ ]:
from prefixspan import PrefixSpan
import os
import pandas as pd
import json

# 将单条记录转换成可区分event
def get_custom_event(event):
    custom_event = ""
    if str(event[0]) == "4":
        return "" # 去除鼠标移动事件
    for idx in range(len(event_indexs)):
        event_idx = event_indexs[idx]
        if event_idx == 7 and str(event[0]) != "7":
            continue
        custom_event = custom_event +"|"+ str(event[event_idx])
    return custom_event

# 区分事件，并对其编号
def number_event(data):
    # custom_event -> number 
    res = {}
    number = 1
    # 记录
    for i in range(len(data)):
        events = data[i]
        # 事件
        for idx in range(len(events)):
            event = events[idx]
            custom_event = get_custom_event(event)
            if custom_event == "":
                continue
            if custom_event in res.keys():
                continue
            res[custom_event] = number
            number = number + 1    
    return res 

event2number = number_event(event_data)
print(event2number)
json_str = json.dumps(event2number, indent=4)
with open('./event2number.json', 'w') as json_file:
    json_file.write(json_str)

In [ ]:
from prefixspan import PrefixSpan
import os
import pandas as pd
import json


# 将事件数据转换成编号序列 [][]
def get_number_data(event_data, event2number):
    res = []
    for i in range(len(event_data)):
        events = event_data[i]
        single_res = []
        # 事件
        last_number = -1 # 将过长的序列 -> 限制长度为5
        cur_number_length = 0
        for idx in range(len(events)):
            event = events[idx]
            custom_event = get_custom_event(event)
            if custom_event in event2number.keys():
                number = event2number[custom_event]
                if number == last_number:
                    cur_number_length = cur_number_length + 1
                    if cur_number_length >= same_seq_length:
                        continue # 截断
                else:
                    cur_number_length = 0
                last_number = number
                single_res.append(number)
        if len(single_res) > 0:
            res.append(single_res)
    return res 

numbers = get_number_data(event_data, event2number)
print(numbers)
with open('./numbers.txt', 'w') as f:
    f.write(str(numbers))

In [5]:
# 将数组按照size划分
def arr_size(arr,size):
    s=[]
    for i in range(0,int(len(arr))+1,size):
        if i+size<=len(arr):
            c=arr[i:i+size]
            s.append(c)
        else:
            c=arr[i:len(arr)]
            s.append(c)
    newlist = [x for x in s if x]
    return newlist

def split_number(numbers):
    res = []
    for idx in range(len(numbers)):
        arrs = arr_size(numbers[idx], max_seq_length)
        for jdx in range(len(arrs)):
            res.append(arrs[jdx])
    return res

db = split_number(numbers)
print(len(db))
with open('./db.txt', 'w') as f:
    f.write(str(db))

13552


In [ ]:
from prefixspan import PrefixSpan
import datetime

def prefixFre(db, fre):
    print("db length=", len(db))
    print("fre = ", fre)

    ps = PrefixSpan(db)
    res = ps.frequent(fre)
    # res = ps.frequent(fre,closed=True)
    print("res length = ",len(res))
    #print(res)
    with open('./res_fre.txt', 'w') as file:
        file.write(str(res))
def prefixTopK(db, k):
    print("db length=", len(db))
    print("k = ", k)

    ps = PrefixSpan(db)
    res = ps.topk(k,closed=True)
    print("res = ",res)
    #print(type(res[0][0]),type(res[0][1]))
    with open('./res_topk.txt', 'w') as file:
        file.write(str(res))

print("prefixSpan run begin. time = ", datetime.datetime.now())
#prefixFre(db, int(len(db) * 0.5))
prefixTopK(db, 1000)
print("prefixSpan run stop. time = ", datetime.datetime.now())
